In [ ]:
#!pip install tkinter
#!pip install termcolor
#!pip install datetime

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import tkinter as tk
from scipy.stats import linregress
from termcolor import colored
import requests, json, datetime, time, sys

In [ ]:
#Straight answer: this is to take in input as a STRING and allow for 
coin_type = input("""What is the name of the coin you are searching for?\nYou can type out the FULL NAME of the coin
or you can use the quick set up. Keep in mind that this option will autofill the currency type.
[1] for Bitcoin (BTC)
[2] for Litecoin (LTC)
[3] for Ethereum (ETH)
""").lower()

In [ ]:
## This is a Dictionary to accept a KEY integer as input (still taken in as a string) and return the VALUE string 
#  attached to it.
quick_keys = {'1': 'bitcoin', '2': 'litecoin', '3': 'ethereum'}

#This is checking for the integer used as a String, and calling the dictionary previously and has 3 or's
if coin_type == '1' or coin_type == "2" or coin_type == '3':
    coin_type = quick_keys[coin_type]
    # I stated in the input, if this IF is TRUE, then CURRENCY TYPE will be autocompleted.
    currency_type = 'usd'
# If the user did not pick an integer, then they will proceed to be asked to continue to input a currency type.
else:
    currency_type = input(colored('What currency type are you looking for: ', 'green')).lower()
                           # ^ WILL LET YOU ATTACH A COLOR TO TEXT WHEN YOU ADD ^

In [ ]:
#Loop to check for an INTEGER input. Will break as soon as the time frame is an integer value
while True:
    #ooo a loop... how do I break it?
    #This try statement will continue so long as the input returns a ValueError... THAT SAID, I WOULD RATHER A CATCH ALL.
    try:
        #This is the TIME FRAME, this will take in how many days prior you would like to look back on.
        time_frame = int(input(colored("How far back would you like to look back?", 'green')))
        #This break will only occur IF AND ONLY IF (IFF) the user gives us an INTEGER and nothing else...
        break
    except ValueError:
        # Don't judge me. I thought this was hilarious at the time... I've been at this for about 10 hours now. 
        # It's awesome.
        print(colored("""                                MISTAKES WERE MADE!                                            

                      Input must be in the form of an INTEGER. Please try again.               """,
                      'red', attrs=['reverse', 'blink']))
        #PLEASE DON'T MESS WITH THE SPACING... THIS WAS 3 MINUTES I'LL NEVER GET BACK.

In [ ]:
url = f'https://api.coingecko.com/api/v3/coins/{coin_type}/market_chart?vs_currency={currency_type}&days={time_frame}'
response = requests.get(url)
print(url)
#I printed this out because I think it's pretty cool how f strings are working for me and I don't have to use the
#.format tool... I love it D: *that's a sadface*

In [ ]:
print(colored(f"""You are looking for {coin_type.capitalize()}'s price, 
in {currency_type.upper()} in a time frame of {time_frame} days""", 'blue'))
#Despite how this will print out, no, it is not clickable... I tried.

In [ ]:
## Oh boy, here comes the code that is harder to understand! Trust me, this is where I spent
## my evening debugging...

#This is the construction of the data for readability. YOU WILL HAVE QUESTIONS, IT'S OKAY TO ASK.
data_construct = response.json()
#This will check to see if the data construction is a dictionary with this and only this as its OUTPUT.
if data_construct == {"error":"Could not find coin with the given id"}:
    print(colored(f"""You goofed.\nThe Currency type: {coin_type} doesn't exist. 
    Please try again with the correct parameters.""", 'red', attrs=['reverse', 'blink']))
#This is another check to see if there is another error and give the correct reason to why the INPUT did not work.
elif data_construct == {'error': 'invalid vs_currency'}:
    print(colored(f"""You goofed.\nThe Currency type: {currency_type} doesn't exist. 
    Please try again with the correct parameters.""", 'red', attrs=['reverse', 'blink']))

In [ ]:
#SO here's the explanation. CoinGecko.com will give us both the time and date together, as a UNIX EPOCH,
#which is normally easy to turn to normal time, but this is in MILISECONDS. Yeah, so when I found out I could
#just divide it by 1000, I was pretty livid. Needless to say, I was still happy that all the outputs work.

#Date_Time will take in a list of the date/time.
date_time = []
#value will take in a list of the values and save them.
value = []
#This is to set a baseline and convert the time to SECONDS from MILISECONDS. Will also allow for correct parameters later
date_time_seconds = []

In [ ]:
#This FOR LOOP will go through the constructed data in the PRICES dataframe, which has 2 columns and separate 
#the two in the date_time and value LISTs. 
for row in data_construct['prices']:
    # (date_time) This one is confusing. First we are calling the variable date_time.
    # (.append) Next we are appending the list date_time
    # (datetime.datetime) This is tricky, don't ask. I'm not sure. I got the info from the library
    #      and this was how it was written.
    #      If I had to guess, it's the DATETIME library, using the DATETIME program... actually, I'm sure it is.
    # (utcfromtimestamp) This is looking at the COORDINATED UNIVERSAL TIME (UTC) function
    # (int(row[0]/1000)) Fun, this had me freaking out for about an hour. Ultimately, you need to ensure that
    #     ROW[0] is an INTEGER and NOT a FLOAT. Let's let that sink in.
    date_time.append(datetime.datetime.utcfromtimestamp(int(row[0]/1000)))
        #So, what this did, was take in all the information from the DATA_CONSTRUCTION variable and change 
        # The entire first element (row[0]) into YEAR/MONTH/DAY.TIME format.
    #This one just took the second element (row[1]) and appended it to value...
    value.append(row[1])
    #This is used to find slope. If you didn't do this, then the plot would be non-existent.
    date_time_seconds.append(int((row[0]/1000)))

In [ ]:
#BEWARE, IF YOU RUN THIS TWICE, THEN YOU WILL GET A PURPLE LINE THAT WILL JUST MESS YOU UP AND MY DATA. STOP IT!

# We want to give it a nice size, not too big though.
plt.figure(figsize=(9,6))
#SO, this was actually really nice to have and allowed the DATES to all appear in the PLOT
plt.xticks(rotation = 30)
#Everyone loves a good GRID.
plt.grid()
##NOTE: I'M NOT ENTIRELY SURE HOW I GOT THIS, BUT IT WORKS. ONLY CHANGE IT IF NOT COMMITTING CHANGES. 
plt.tight_layout(rect=[0, 0.05, 1, 0.95])
#Time to make our PLOT
plt.plot(date_time, value, color = 'black')
#Give the thing a good TITLE
plt.title(f"Bitcoin price over {time_frame} days")
#and an X-LABEL
plt.xlabel('Date')
#Maybe we'll see it, probably not, but here's a Y-LABEL
plt.ylabel('Price per Coin')
#This is to plot LINEAR REGRESSION. This is where I found the real problem with the code and had to create 
#several work arounds in order to figure out the solution. Luckily, we had the best Hookah ever!
#NOTE: THIS CURRENTLY WORKS. DO NOT ATTEMPT TO MAKE CHANGES WITHOUT CHECKING DOCUMENTATION! REMEMBER: DOCUMENTATION!!!
regression = []
(slope, intercept, rvalue, pvalue, stderr) = linregress(date_time_seconds, value)
regress_values = []
for i in date_time_seconds:
    regress_values.append(i * slope + intercept)
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(date_time, regress_values,'-.', color = 'red')

In [ ]:
pd.read_html('https://finance.yahoo.com/quote/%5EDJI/history/')[0]

In [ ]:

stock_market = pd.read_csv('stockmarket.csv')
stock_market


In [ ]:
stock_market['Date'] = pd.to_datetime(stock_market['Date'])

stock_market.dtypes

In [ ]:
crypto_df = pd.DataFrame ( { "date" : date_time,
                      "price" : value })
                         
crypto_df

In [ ]:
 merged_df = pd.merge(crypto_df, stock_market, how= 'right', left_on= 'date' , right_on = 'Date' ).dropna()
 merged_df


In [ ]:

merged_df[['Date', 'price', 'Open']].plot(x= "Date", figsize = (9,6)).grid()

try:
    os.remove("Jawaher.png")
    print("File exists, deleting.")
    plt.savefig('Jawaher.png')
except:
    print("Creating save of Jawaher.png")
    plt.savefig('Jawaher.png')
    pass
